# Southampton Tidal Curve Interactive Viewer

Interactive tidal height selection on scales and tidal hour selection with dynamic purple and yellow lines.

In [ ]:
from IPython.display import display, HTML

html_content = '''
<style>
  #container {
    position: relative;
    width: 1100px;
    margin: 40px auto;
  }
  canvas {
    border: 1px solid #aaa;
    display: block;
    background: #fff;
  }
</style>
<div id="container">
  <canvas id="tidal-canvas" width="1100" height="570"></canvas>
</div>

<script>
const canvas = document.getElementById('tidal-canvas');
const ctx = canvas.getContext('2d');
const img = new Image();
img.src = '2025-Scan-of-Southampton-Tidal-Curve.jpg';

const graph = {
  top: 55, bottom: 480, left: 110, right: 1020,
  width: 910, height: 425,
  maxHW: 5.0, maxLW: 3.0,
  hourCount: 12
};

const lwScaleZone = { x: 90, yTop: 460, yBottom: 520, width: 40 };
const hwScaleZone = { x: 90, yTop: 40, yBottom: 100, width: 40 };
const tidalHourZone = { yTop: 490, yBottom: 520, xLeft: graph.left, xRight: graph.right };

let lowWaterHeight = null;
let highWaterHeight = null;
let selectedHourPos = null;

function heightToY(height, maxHeight) {
  return graph.bottom - (height / maxHeight) * (graph.bottom - graph.top);
}

function hourToX(hourFrac) {
  return graph.left + (hourFrac / 10) * graph.width;
}

function xToHourFrac(x) {
  const clampedX = Math.min(Math.max(x, graph.left), graph.right);
  const frac = (clampedX - graph.left) / graph.width * 10.0;
  return Math.round(frac * 4) / 4;
}

function draw() {
  ctx.clearRect(0, 0, canvas.width, canvas.height);
  ctx.drawImage(img, 0, 0, canvas.width, canvas.height);

  if (lowWaterHeight !== null) {
    const y = heightToY(lowWaterHeight, graph.maxLW);
    const x = lwScaleZone.x + lwScaleZone.width / 2;
    drawDot(x, y, 'red');
  }
  if (highWaterHeight !== null) {
    const y = heightToY(highWaterHeight, graph.maxHW);
    const x = hwScaleZone.x + hwScaleZone.width / 2;
    drawDot(x, y, 'red');
  }

  if (lowWaterHeight !== null && highWaterHeight !== null) {
    ctx.beginPath();
    ctx.moveTo(lwScaleZone.x + lwScaleZone.width / 2, heightToY(lowWaterHeight, graph.maxLW));
    ctx.lineTo(hwScaleZone.x + hwScaleZone.width / 2, heightToY(highWaterHeight, graph.maxHW));
    ctx.lineWidth = 3;
    ctx.strokeStyle = 'purple';
    ctx.stroke();

    if (selectedHourPos !== null) {
      drawYellowLines(selectedHourPos);
    }
  }
}

function drawDot(x, y, color) {
  ctx.beginPath();
  ctx.arc(x, y, 6, 0, 2 * Math.PI);
  ctx.fillStyle = color;
  ctx.fill();
  ctx.lineWidth = 2;
  ctx.strokeStyle = 'black';
  ctx.stroke();
}

function drawYellowLines(xPos) {
  const hourFrac = xToHourFrac(xPos);
  const lowY = heightToY(lowWaterHeight, graph.maxLW);
  const highY = heightToY(highWaterHeight, graph.maxHW);
  const curveY = lowY - (hourFrac / 10) * (lowY - highY);

  ctx.beginPath();
  ctx.moveTo(xPos, tidalHourZone.yBottom);
  ctx.lineTo(xPos, curveY);
  ctx.strokeStyle = 'yellow';
  ctx.lineWidth = 3;
  ctx.stroke();

  let t = (curveY - lowY) / (highY - lowY);
  if (!isFinite(t)) t = 0;
  const intersectX = (hwScaleZone.x + hwScaleZone.width / 2) * t + (lwScaleZone.x + lwScaleZone.width / 2) * (1 - t);

  ctx.beginPath();
  ctx.moveTo(xPos, curveY);
  ctx.lineTo(intersectX, curveY);
  ctx.stroke();

  ctx.beginPath();
  ctx.moveTo(intersectX, curveY);
  ctx.lineTo(intersectX, heightToY(highWaterHeight, graph.maxHW));
  ctx.stroke();
}

canvas.addEventListener('mousedown', (e) => {
  const rect = canvas.getBoundingClientRect();
  const mx = e.clientX - rect.left;
  const my = e.clientY - rect.top;

  if (mx >= lwScaleZone.x && mx <= lwScaleZone.x + lwScaleZone.width &&
      my >= lwScaleZone.yTop && my <= lwScaleZone.yBottom) {
    let rel = (lwScaleZone.yBottom - my) / (lwScaleZone.yBottom - lwScaleZone.yTop);
    lowWaterHeight = Math.min(Math.max(rel * graph.maxLW, 0), graph.maxLW);
    draw();
    return;
  }

  if (mx >= hwScaleZone.x && mx <= hwScaleZone.x + hwScaleZone.width &&
      my >= hwScaleZone.yTop && my <= hwScaleZone.yBottom) {
    let rel = (hwScaleZone.yBottom - my) / (hwScaleZone.yBottom - hwScaleZone.yTop);
    highWaterHeight = Math.min(Math.max(rel * graph.maxHW, 0), graph.maxHW);
    draw();
    return;
  }

  if (lowWaterHeight !== null && highWaterHeight !== null) {
    if (mx >= tidalHourZone.xLeft && mx <= tidalHourZone.xRight &&
        my >= tidalHourZone.yTop && my <= tidalHourZone.yBottom) {
      selectedHourPos = hourToX(xToHourFrac(mx));
      draw();
      return;
    }
  }
});

img.onload = () => {
  draw();
};
</script>
'''
display(HTML(html_content))